# Loading the data

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import feather
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', 500)

In [8]:
#Loading Train and Test Data
train = feather.read_dataframe("../input/feather/train.feather")
test = feather.read_dataframe("../input/feather/test.feather")
train['first_active_month'] = pd.to_datetime(train['first_active_month'])
train['elapsed_time'] = (datetime.date(2018, 2, 1) - train['first_active_month'].dt.date).dt.days
test['first_active_month'] = pd.to_datetime(test['first_active_month'])
test['elapsed_time'] = (datetime.date(2018, 2, 1) - test['first_active_month'].dt.date).dt.days

In [2]:
# transactions data
new_transactions = feather.read_dataframe('../input/feather/new_merchant_transactions.feather')
historical_transactions =feather.read_dataframe('../input/feather/historical_transactions.feather')

def binarize(df):
    for col in ['authorized_flag', 'category_1']:
        df[col] = df[col].map({'Y':1, 'N':0})
    return df

historical_transactions = binarize(historical_transactions)
new_transactions = binarize(new_transactions)

In [8]:
agg_fun = {'authorized_flag': ['mean']}
auth_mean = historical_transactions.groupby(['card_id']).agg(agg_fun)
auth_mean.columns = ['_'.join(col).strip() for col in auth_mean.columns.values]
auth_mean.reset_index(inplace=True)

In [13]:
# 桁数
len(str(auth_mean["authorized_flag_mean"][1]))

17

In [23]:
authorized_transactions = historical_transactions[historical_transactions['authorized_flag'] == 1]
historical_transactions = historical_transactions[historical_transactions['authorized_flag'] == 0]

In [24]:
authorized_transactions

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,merchant_label_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_e020e9b302,292910.0,-8,-0.703331,2017-06-25 15:33:07+00:00,1.0,16,37
1,1,C_ID_4e6213e9bc,88,0,0,A,367,M_ID_86ec983688,176216.0,-7,-0.733128,2017-07-15 12:10:45+00:00,1.0,16,16
2,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_979ed661fc,198285.0,-6,-0.720386,2017-08-09 22:04:29+00:00,1.0,16,37
3,1,C_ID_4e6213e9bc,88,0,0,A,560,M_ID_e6d5ae8ea6,301748.0,-5,-0.735352,2017-09-02 10:06:26+00:00,1.0,16,34
4,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_e020e9b302,292910.0,-11,-0.722865,2017-03-10 01:14:19+00:00,1.0,16,37
5,1,C_ID_4e6213e9bc,333,0,0,A,80,M_ID_50af771f8d,105279.0,0,-0.734887,2018-02-24 08:45:05+00:00,1.0,9,37
6,1,C_ID_4e6213e9bc,88,0,0,A,278,M_ID_5e8220e564,123319.0,-11,-0.716855,2017-03-21 00:10:51+00:00,1.0,16,37
7,1,C_ID_4e6213e9bc,3,0,0,A,80,M_ID_9d41786a50,205584.0,-3,-0.657049,2017-11-18 20:05:55+00:00,1.0,16,37
8,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_979ed661fc,198285.0,-8,-0.737967,2017-06-01 22:02:56+00:00,1.0,16,37
9,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_74ba14b5fc,152426.0,-11,-0.715352,2017-03-16 15:41:22+00:00,1.0,16,37
